In [ ]:
#Running on Colab
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=51f65a301a5cc2eae6ef174bf2b6ec8393cf67eb6faa88bf305febf39cd1be7e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [ ]:
#  Importing Required Libraries
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# Create Spark session and ContextRun PySpark.
# create the session
conf = SparkConf().set("spark.ui.port","4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.appName("DataFrame").config('spark.ui.port', '4050').getOrCreate()
spark

## **Davies-Bouldin index (DBI)**

In [ ]:
def getPointInfo(line):
  points,clusterID=line.split("), ")
  pointX,pointY=points.split(',')
  pointX=float(pointX.replace('(',''))
  pointY=float(pointY)

  return clusterID,(pointX,pointY,1)

clustersData=sc.textFile("F1.txt").map(lambda line: getPointInfo(line))
clustersData.collect()

[('C1', (1.0, 2.0, 1)),
 ('C2', (5.0, 5.0, 1)),
 ('C1', (2.0, 2.0, 1)),
 ('C3', (11.0, 11.0, 1))]

In [ ]:
import numpy as np
centroids=clustersData.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1],x[2]+y[2])).mapValues(lambda x:(x[0]/x[2],x[1]/x[2]))
centroids.collect()

[('C1', (1.5, 2.0)), ('C2', (5.0, 5.0)), ('C3', (11.0, 11.0))]

In [ ]:
centroidsPointsDist=centroids.join(clustersData).mapValues(lambda x: (((x[0][0]-x[1][0])**2+(x[0][1]-x[1][1])**2),1)).reduceByKey(lambda x,y:((np.sqrt(x[0])+np.sqrt(y[0])),(x[1]+y[1])))
centroidsPointsDist=centroidsPointsDist.mapValues(lambda x: x[0]/x[1])
centroidsPointsDist.collect()

[('C1', 0.5), ('C2', 0.0), ('C3', 0.0)]

In [ ]:
centoidsDict=dict(centroids.collect())
centroidsPointsDistDict=dict(centroidsPointsDist.collect())
results={}
for key,value in centoidsDict.items():
  for key2,value2 in centoidsDict.items():
    if(key!=key2 and (str(key)+str(key2) not in results.keys()) and (str(key2)+str(key) not in results.keys())):
      newPair=str(key)+str(key2)
      euclideanDistanceBetweenCentroid = np.sqrt((value[0]-value2[0])**2+(value[1]-value2[1])**2)
      maxDist=centroidsPointsDistDict[key] if centroidsPointsDistDict[key]>centroidsPointsDistDict[key2] else centroidsPointsDistDict[key2]

      results[newPair]=euclideanDistanceBetweenCentroid/maxDist if maxDist > 0 else 0

In [ ]:
results

{'C1C2': 9.219544457292887, 'C1C3': 26.1725046566048, 'C2C3': 0}

In [ ]:
sumVal,countVal=0,0
for key, value in results.items():
  sumVal=sumVal+value
  countVal=countVal+1
dbi=sumVal/countVal if countVal>0 else 0
print("DBI=",dbi)

DBI= 11.79734970463256


ALL TOGETHER

In [ ]:
import numpy as np
def getPointInfo(line):
  points,clusterID=line.split("), ")
  pointX,pointY=points.split(',')
  pointX=float(pointX.replace('(',''))
  pointY=float(pointY)

  return clusterID,(pointX,pointY,1)

clustersData=sc.textFile("F1.txt").map(lambda line: getPointInfo(line))
clustersData.collect()


centroids=clustersData.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1],x[2]+y[2])).mapValues(lambda x:(x[0]/x[2],x[1]/x[2]))


centroidsPointsDist=centroids.join(clustersData).mapValues(lambda x: (((x[0][0]-x[1][0])**2+(x[0][1]-x[1][1])**2),1)).reduceByKey(lambda x,y:((np.sqrt(x[0])+np.sqrt(y[0])),(x[1]+y[1])))
centroidsPointsDist=centroidsPointsDist.mapValues(lambda x: x[0]/x[1])


centoidsDict=dict(centroids.collect())
centroidsPointsDistDict=dict(centroidsPointsDist.collect())
results={}
for key,value in centoidsDict.items():
  for key2,value2 in centoidsDict.items():
    if(key!=key2 and (str(key)+str(key2) not in results.keys()) and (str(key2)+str(key) not in results.keys())):
      newPair=str(key)+str(key2)
      euclideanDistanceBetweenCentroid = np.sqrt((value[0]-value2[0])**2+(value[1]-value2[1])**2)
      maxDist=centroidsPointsDistDict[key] if centroidsPointsDistDict[key]>centroidsPointsDistDict[key2] else centroidsPointsDistDict[key2]

      results[newPair]=euclideanDistanceBetweenCentroid/maxDist if maxDist > 0 else 0

sumVal,countVal=0,0
for key, value in results.items():
  sumVal=sumVal+value
  countVal=countVal+1
dbi=sumVal/countVal if countVal>0 else 0
print("DBI=",dbi)

DBI= 11.79734970463256


**SAME CODE USING DATAFRAMES**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, udf, sum as spark_sum, count as spark_count
from pyspark.sql.types import StructType, StructField, FloatType, StringType, IntegerType
import numpy as np

# Define schema for input data
schema = StructType([
    StructField("points", StringType(), True),
    StructField("clusterID", StringType(), True)
])

# Read data into DataFrame
data = spark.read.option("delimiter", "), ").schema(schema).csv("F1.txt")

# Extract x, y coordinates from points and create new DataFrame
points_df = data.withColumn("pointX", split(col("points"), ",").getItem(0).substr(2, 100).cast(FloatType())) \
                .withColumn("pointY", split(col("points"), ",").getItem(1).cast(FloatType()))

# Calculate centroids
centroids = points_df.groupBy("clusterID") \
                     .agg(spark_sum("pointX").alias("sumX"), spark_sum("pointY").alias("sumY"), spark_count("*").alias("count")) \
                     .withColumn("centroidX", col("sumX") / col("count")) \
                     .withColumn("centroidY", col("sumY") / col("count"))

# Calculate distances from points to centroids
joined_df = points_df.join(centroids, "clusterID") \
                     .withColumn("distance", ((col("centroidX") - col("pointX"))**2 + (col("centroidY") - col("pointY"))**2)**0.5)

# Calculate average distance to centroid for each cluster
average_distances = joined_df.groupBy("clusterID") \
                             .agg(spark_sum("distance").alias("sumDistance"), spark_count("*").alias("count")) \
                             .withColumn("avgDistance", col("sumDistance") / col("count"))

# Convert centroids and average distances to dictionaries
centroids_dict = {row['clusterID']: (row['centroidX'], row['centroidY']) for row in centroids.collect()}
average_distances_dict = {row['clusterID']: row['avgDistance'] for row in average_distances.collect()}

# Calculate DBI
results = {}
for key, value in centroids_dict.items():
    for key2, value2 in centroids_dict.items():
        if key != key2 and (str(key) + str(key2) not in results) and (str(key2) + str(key) not in results):
            euclidean_distance = np.sqrt((value[0] - value2[0])**2 + (value[1] - value2[1])**2)
            max_dist = average_distances_dict[key] if average_distances_dict[key]>average_distances_dict[key2] else average_distances_dict[key2]
            results[str(key) + str(key2)] = euclidean_distance / max_dist if max_dist > 0 else 0

# Initialize variables for DBI calculation
sum_val = 0
count_val = 0

# Calculate DBI using results dictionary
for value in results.values():
    sum_val += value
    count_val += 1

# Avoid division by zero error
dbi = sum_val / count_val if count_val > 0 else 0

# Print DBI value
print("DBI =", dbi)

DBI = 11.79734970463256
